In [1]:
import pandas as pd
import numpy as np
import gzip, csv
import dask.dataframe as dd
import io
from time_diff import time_diff
import array

In [2]:
labevents_df = pd.read_csv('/Volumes/Kash_WB/MIMICIII/Data/LABEVENTS.csv.gz', compression='gzip')
d_labitems_df = pd.read_csv('/Volumes/Kash_WB/MIMICIII/Data/D_LABITEMS.csv.gz', compression='gzip')
outputevents_df = pd.read_csv('/Volumes/Kash_WB/MIMICIII/Data/OUTPUTEVENTS.csv.gz', compression='gzip')
d_items_df = pd.read_csv('/Volumes/Kash_WB/MIMICIII/Data/D_ITEMS.csv.gz', compression='gzip')
icustays_df = pd.read_csv('/Volumes/Kash_WB/MIMICIII/Data/ICUSTAYS.csv.gz', compression='gzip')
patients_df = pd.read_csv('/Volumes/Kash_WB/MIMICIII/Data/PATIENTS.csv.gz', compression='gzip')

In [3]:
icd_df = pd.read_csv('/Volumes/Kash_WB/MIMICIII/Data/DIAGNOSES_ICD.csv.gz', compression='gzip')
icd_np = np.array(icd_df)

In [4]:
labevents_np = np.array(labevents_df)
outputevents_np = np.array(outputevents_df)

In [5]:
gcs = [723, 454, 184, 223900, 223901, 220739] #0, 1, 2
bp = [51, 442, 455, 6701, 220179, 220050] #3, 4, 5 
hr = [211, 220045]                        #6, 7, 8
bt = [678, 223761, 676, 223762]           #9, 10, 11
pao = [50821, 50816]                      # 12, 13, 14 
fio = [223835, 3420, 3422, 190]         #15, 16, 17
# pao_fio                               #18 
uo = [40055, 43175, 40069, 40094, 40715, 40473, 40085, 40057, 40056, 40405, 40428, 40086, 40096,
     40651, 226559, 226560, 226561, 226584, 226563, 226564, 226565, 226567, 226557, 225558, 227488, 227489] #19, 20, 21
un = [51006]                              #22, 23, 24
wbc = [51300, 51301]                      #25, 26, 27
bic = [50882]                             #28, 29, 30
na = [950824, 50983]                      #31, 32, 33
k = [50822, 50971]                        #34, 35, 36
bili = [50885]                            #37, 38, 39

# Extracting Features from Outputevents

In [6]:
output_arr = [0] * 50
output_dict = {}

In [7]:
#keys dont include nan and itemids are included in the feature list
for row in outputevents_np:
    if str(row[2]).endswith('.0') and str(row[6]) != 'nan':
            hamid = int(row[2])
            itemid = row[5]
            value = row[6]
            time = row[4]

            if hamid not in output_dict.keys():
                if itemid in uo:
                    output_arr[19] = itemid
                    output_arr[20] = value
                    output_arr[21] = time
                    output_dict[hamid] = output_arr
            elif hamid in output_dict.keys():
                existing_time = output_dict[hamid][21]
                timediff = time_diff(existing_time, time)
                if timediff > 0:
                    pass
                if timediff < 0:
                    #overwrite the existing values in array with the earliest itemid values
                    output_dict[hamid][19] = itemid
                    output_dict[hamid][20] = value
                    output_dict[hamid][21] = time
    output_arr = [0] * 50

In [8]:
with open('output3.csv', 'w') as csv_file:
    writer = csv.writer(csv_file)
    for key, value in output_dict.items():
       writer.writerow([key, value])

In [10]:
len(output_dict.keys())

51090

# Extracting Features from Labevents

In [14]:
lab_arr = [0] * 50
lab_dict = output_dict

In [13]:
labevents_df.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ITEMID,CHARTTIME,VALUE,VALUENUM,VALUEUOM,FLAG
0,281,3,NaN,50820,2101-10-12 16:07:00,7.39,7.39,units,NaN
1,282,3,NaN,50800,2101-10-12 18:17:00,ART,NaN,NaN,NaN
2,283,3,NaN,50802,2101-10-12 18:17:00,-1,-1.00,mEq/L,NaN
3,284,3,NaN,50804,2101-10-12 18:17:00,22,22.00,mEq/L,NaN
4,285,3,NaN,50808,2101-10-12 18:17:00,0.93,0.93,mmol/L,abnormal


In [15]:
for row in labevents_np:
    if str(row[2]).endswith('.0'):
        hamid = int(row[2])
        itemid = row[3]
        time = row[4]
        value = row[5]

        if hamid not in lab_dict.keys():
            if itemid in pao:
                lab_arr[12] = itemid
                lab_arr[13] = value
                lab_arr[14] = time
            if itemid in un:
                lab_arr[22] = itemid
                lab_arr[23] = value
                lab_arr[24] = time
            if itemid in wbc:
                lab_arr[25] = itemid
                lab_arr[26] = value
                lab_arr[27] = time
            if itemid in bic:
                lab_arr[28] = itemid
                lab_arr[29] = value
                lab_arr[30] = time
            if itemid in na:
                lab_arr[31] = itemid
                lab_arr[32] = value
                lab_arr[33] = time
            if itemid in k:
                lab_arr[34] = itemid
                lab_arr[35] = value
                lab_arr[36] = time
            if itemid in bili:
                lab_arr[37] = itemid
                lab_arr[38] = value
                lab_arr[39] = time
                
                lab_dict[hamid] = lab_arr

        elif hamid in lab_dict.keys():
            if itemid in lab_dict[hamid]:
                existing_hamid_value = lab_dict[hamid]
                itemid_index = existing_hamid_value.index(itemid)
                value_index = itemid_index + 1
                time_index = itemid_index + 2
                existing_time = existing_hamid_value[time_index]
                timediff = time_diff(existing_time, time)
                if timediff > 0:
                    continue
                elif timediff < 0:
                    #overwrite the existing values in array with the earliest itemid values
                    lab_dict[hamid][itemid_index] = itemid
                    lab_dict[hamid][value_index] = value
                    lab_dict[hamid][time_index] = time
                    
            elif itemid not in lab_dict[hamid]:
                if itemid in pao:
                    lab_dict[hamid][12] = itemid
                    lab_dict[hamid][13] = value
                    lab_dict[hamid][14] = time
                if itemid in un:
                    lab_dict[hamid][22] = itemid
                    lab_dict[hamid][23] = value
                    lab_dict[hamid][24] = time
                if itemid in wbc:
                    lab_dict[hamid][25] = itemid
                    lab_dict[hamid][26] = value
                    lab_dict[hamid][27] = time
                if itemid in bic:
                    lab_dict[hamid][28] = itemid
                    lab_dict[hamid][29] = value
                    lab_dict[hamid][30] = time
                if itemid in na:
                    lab_dict[hamid][31] = itemid
                    lab_dict[hamid][32] = value
                    lab_dict[hamid][33] = time
                if itemid in k:
                    lab_dict[hamid][34] = itemid
                    lab_dict[hamid][35] = value
                    lab_dict[hamid][36] = time
                if itemid in bili:
                    lab_dict[hamid][37] = itemid
                    lab_dict[hamid][38] = value
                    lab_dict[hamid][39] = time
    
    lab_arr = [0] * 50

In [149]:
print(len(lab_dict.keys()))

55849


In [18]:
with open('lab2.csv', 'w') as csv_file:
    writer = csv.writer(csv_file)
    for key, value in lab_dict.items():
       writer.writerow([key, value])

In [19]:
lab_dict

{177991: [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  50816,
  '40',
  '2142-09-04 17:37:00',
  0,
  0,
  0,
  0,
  40055,
  50.0,
  '2142-08-30 16:00:00',
  51006,
  '81',
  '2142-08-30 19:00:00',
  51301,
  '14.2',
  '2142-08-30 19:00:00',
  50882,
  '30',
  '2142-08-30 19:00:00',
  50983,
  '145',
  '2142-08-30 19:00:00',
  50971,
  '4.2',
  '2142-09-04 06:22:00',
  50885,
  '0.3',
  '2142-08-30 19:00:00',
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 143418: [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  50821,
  '115',
  '2144-07-09 16:44:00',
  0,
  0,
  0,
  0,
  40069,
  80.0,
  '2144-07-09 19:15:00',
  51006,
  '15',
  '2144-07-08 21:10:00',
  51301,
  '14.8',
  '2144-07-08 21:10:00',
  50882,
  '18',
  '2144-07-08 21:10:00',
  50983,
  '133',
  '2144-07-08 21:10:00',
  50971,
  '4.4',
  '2144-07-09 05:00:00',
  50885,
  '0.4',
  '2144-07-10 06:00:00',
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 126869: [0,
  0,
  0,
  0,
  0,
  0,
 

# Extracting Features from Diagnosis_ICD 

In [154]:
hiv = ["42", "7953", "79571", "V6544"]
hematological = ["7768", "7769", "67801", "67803"]
cancer = ["3614"]

In [141]:
diag_arr = [0] * 50

In [142]:
icd_df.head()

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,1297,109,172335,1.0,40301
1,1298,109,172335,2.0,486
2,1299,109,172335,3.0,58281
3,1300,109,172335,4.0,5855
4,1301,109,172335,5.0,4254


In [165]:
for row in icd_np:
    hamid = int(row[2])
    icd_code = row[4]
    if hamid in lab_dict.keys():
        if icd_code in hiv:
            lab_dict[hamid][43] = icd_code
            lab_dict[hamid][44] = val
        if icd_code in hematological:
            lab_dict[hamid][45] = icd_code
            lab_dict[hamid][46] = val
        if icd_code in cancer:
            lab_dict[hamid][47] = icd_code
            lab_dict[hamid][48] = val
    

In [152]:
with open('lab3.csv', 'w') as csv_file:
    writer = csv.writer(csv_file)
    for key, value in lab_dict.items():
       writer.writerow([key, value])

In [53]:
for row in icd_np:
    #if str(row[2]).endswith('.0'):
    hamid = int(row[2])
    icd_code = row[4]

    if hamid in diag_dict.keys():
        if icd_code not in diag_dict[hamid]:
            if icd_code in hiv:
                diag_dict[hamid][43] = icd_code
                diag_dict[hamid][44] = 19
            elif icd_code in hematological:
                diag_dict[hamid][45] = icd_code
                diag_dict[hamid][46] = 10
            elif icd_code in cancer:
                diag_dict[hamid][47] = icd_code
                diag_dict[hamid][48] = 9
    
    
    elif hamid not in diag_dict.keys():
        if icd_code in hiv:
            diag_arr[43] = icd_code
            diag_arr[44] = 19
            diag_dict[hamid] = diag_arr
        elif icd_code in hematological:
            diag_arr[45] = icd_code
            diag_arr[46] = 10
            diag_dict[hamid] = diag_arr
        elif icd_code in cancer:
            diag_arr[47] = icd_code
            diag_arr[48] = 9
            diag_dict[hamid] = diag_arr
    diag_arr = [0] * 50

In [54]:
with open('diag2.csv', 'w') as csv_file:
    writer = csv.writer(csv_file)
    for key, value in diag_dict.items():
       writer.writerow([key, value[43:49]])

In [33]:
arr = [0, 0]
x = {'a': [23, 444], 'b': [388, 909]}

In [34]:
arr[1] = 83
x['a'] = arr

In [52]:
x['b']

[388, 909]

# Extracting Features from Admissions

In [ ]:
admission_df = pd.read_csv("/Volumes/Kash_WB/MIMICIII/Data/ADMISSIONS.csv.gz", compression='gzip')
admission_np = np.array(admission_df)

In [ ]:
admiss_dict = {'EMERGENCY': 8, 'URGENT': 8, 'NEWBORN' : 0, 'ELECTIVE': 6}

In [ ]:
admin_arr = [0] * 55

In [ ]:
for row in admission_np:
    if isinstance(row[2], int):
        hamid = row[2]
    
        admiss_type = row[6]
        admiss_id = admiss_dict[admiss_type] 

        if hamid in diag_dict.keys():
            diag_dict[hamid][53] = admiss_id
        
        if hamid not in diag_dict.keys():
            admin_arr[53] = admiss_id
            diag_dict[hamid] = admin_arr
    

In [ ]:
diag_dict

# Feature Extraction from Chartevents

In [ ]:
diff = 0

In [ ]:


for k,v in diag_dict.items():
    if v == v:
        diff = diff +1
        
        

In [ ]:
print(diff)

In [ ]:
with open('untitled.csv', 'w') as csv_file:
    writer = csv.writer(csv_file)
    for key, value in diag_dict.items():
       writer.writerow([key, value])

# Calculating age from ICUstays and Admissions

In [ ]:
patients_np = np.array(patients_df)

In [ ]:
icustays_np = np.array(icustays_df)

In [ ]:
temp_dict = diag_dict

In [ ]:
for k,v in diag_dict.items():
    v[40] = 0
    v[41] = 0
    v[42] = 0 

In [ ]:
diag_dict

In [ ]:
icustays_df.head()

In [ ]:
for row in icustays_np:
    hamid = row[2]
    intime = row[9]
    
    if hamid in diag_dict.keys():
        diag_dict[hamid][40] = intime

In [ ]:
patients_df.head()

# subid mapping to hadmid

In [ ]:
admission_dict = {}
patient_dict = {}

In [ ]:
for row in patients_np:
    subid = row[1]
    dob = row[3]
    
    if subid not in patient_dict.keys():
        patient_dict[subid] = dob

In [ ]:
for row in admission_np:
    subid = row[1]
    hamid = row[2]
    admittime = row[3]
    
    if subid not in admission_dict.keys():
        admission_dict[subid] = [admittime, hamid]
    
    if subid in admission_dict.keys():
        if hamid not in admission_dict[subid]:
            admission_dict[subid].append(hamid)
        
        timediff = time_diff(admission_dict[subid][0], admittime)
        if timediff > 0:
            pass
        elif timediff < 0:
    
            admission_dict[subid][0] = admittime

In [ ]:
diag_dict

In [ ]:
patient_dict

In [ ]:
admission_dict

In [ ]:
admission_dict[357][1:]

In [ ]:
diag_dict

In [ ]:
print(len(diag_dict.keys()))

In [ ]:
for (admiss_k, admiss_v), (diag_k, diag_v) in zip(admission_dict.items(), diag_dict.items()):
    subid = admiss_k
    hamidS = admiss_v[1:]
    hamid = diag_k
    if hamid in hamidS:
        diag_v[41] = patient_dict[admiss_k]
        

In [ ]:
{k:v for k,v in temp_dict.items() if v[40:42] != 0}

In [ ]:
print(len(diag_dict.keys()))

In [ ]:
for key, val in diag_dict.items():
    intime = val[40]
    dob = val[41]
    if intime != 0 and dob != 0:
        timediff = time_diff(dob, intime)
        
        val[42] = timediff

In [ ]:
diag_dict

In [ ]:
val = []

{val.append(v) for k,v in diag_dict.items() if v[42] >= 12198.166666666666}

print(len(val))

In [ ]:
print(len(diag_dict.keys()))